# Forward differentiation

See

- [Simple forward-mode AD in Julia using Dual Numbers and Operator Overloading](https://www.youtube.com/watch?v=DfKKTE9XU0Q&list=PLISXH-iEM4JkHNH8Ji0ELDYcak7FD5gqa)

- [Simple (scalar) forward-mode Automatic Differentiation with DualNumbers and operator overloading](https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/adjoints_sensitivities_automatic_differentiation/scalar_forward_ad_with_dual_numbers.jl)



The **dual numbers** are expressions of the form $r + ε d$, where $r$ and $d$ are real numbers, and $ε$ is a symbol taken to satisfy $ε^2 = 0$ and $ε \neq 0$.

- $(r_1 + ε d_1) + (r_2 + ε d_2) = (r_1 + r_2) + ε (d_1 + d_2)$
- $(r_1 + ε d_1) * (r_2 + ε d_2) = (r_1 * r_2) + ε (r_1 * d_2 + d_1 * r_2)$

In [1]:
struct DualNumber{T<:Real}
    r::T
    d::T
end

In [2]:
f(x) = x * x
∇f(x) = 2.0 * x

∇f (generic function with 1 method)

In [3]:
f(3.0), ∇f(3.0)

(9.0, 6.0)

In [4]:
function derivative(f, x::Real)
    f(DualNumber(x, one(x))).d
end

derivative (generic function with 1 method)

In [5]:
try derivative(f, 3.0) catch e; e end

MethodError(*, (DualNumber{Float64}(3.0, 1.0), DualNumber{Float64}(3.0, 1.0)), 0x000000000000685c)

$(f\,g)' = f\, g' + f'\,g$

In [6]:
import Base: +, *, /, sin

In [7]:
*(x::DualNumber, y::DualNumber) =
    DualNumber(x.r * y.r, x.r * y.d + x.d * y.r)
*(x::Real, y::DualNumber) =
    DualNumber(x, zero(x)) * y

* (generic function with 223 methods)

In [8]:
derivative(f, 3.0)

6.0

In [9]:
g(x) = 3.0 * x * x + sin(x)
∇g(x) = 6.0 * x + cos(x)

g(3.0), ∇g(3.0)

(27.14112000805987, 17.010007503399553)

In [10]:
try derivative(g, 3.0) catch e; e end

MethodError(sin, (DualNumber{Float64}(3.0, 1.0),), 0x0000000000006860)

- $(\sin x)' = \cos x$
- $y = f(g(x)) ⇒ y' = f'(g(x))\,g'(x) $

In [11]:
sin(x::DualNumber) = DualNumber(sin(x.r), cos(x.r) * x.d)

sin (generic function with 15 methods)

In [12]:
try derivative(g, 3.0) catch e; e end

MethodError(+, (DualNumber{Float64}(27.0, 18.0), DualNumber{Float64}(0.1411200080598672, -0.9899924966004454)), 0x0000000000006861)

$(f + g)' = f' + g'$

In [13]:
+(x::DualNumber, y::DualNumber) =
    DualNumber(x.r + y.r, x.d + y.d)
+(x::Real, y::DualNumber) =
    DualNumber(x, zero(x)) + y

+ (generic function with 200 methods)

In [14]:
derivative(g, 3.0)

17.010007503399553

See

- [Automatic Differentiation in 10 minutes with Julia](https://www.youtube.com/watch?v=vAp6nUMrKYg)

## Babylonian sqrt

Repeat $t ← (t + x/t)/2$ until $t$ converges to $\sqrt x$.

In [15]:
function babylonian(x; N=10)
    t = (1.0 + x) / 2.0
    for i = 2:N
        t = (t + x / t) / 2.0
    end
    t
end

babylonian (generic function with 1 method)

In [16]:
sqrt(2.0), babylonian(2.0)

(1.4142135623730951, 1.414213562373095)

In [17]:
∇sqrt(x) = 0.5 / sqrt(x)
∇sqrt(2.0)

0.35355339059327373

In [18]:
try derivative(babylonian, 2.0) catch e; e end

MethodError(/, (DualNumber{Float64}(3.0, 1.0), 2.0), 0x0000000000006867)

$(f/g)' = (f'g - fg')/g^2$

In [19]:
/(x::DualNumber, y::DualNumber) =
    DualNumber(x.r / y.r, (x.d * y.r - x.r * y.d) / (y.r * y.r))
/(x::DualNumber, y::Real) =
    x / DualNumber(y, zero(y))

/ (generic function with 131 methods)

In [20]:
derivative(babylonian, 2.0), ∇sqrt(2.0)

(0.35355339059327373, 0.35355339059327373)